In [2]:
import requests
from bs4 import BeautifulSoup
import csv

urls = []

for x in range(1, 30):
    url = "https://visitseattle.org/events/page/" + str(x)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    selector = "div.search-result-preview > div > h3 > a"
    a_eles = soup.select(selector)
    urls.extend([x['href'] for x in a_eles])

def get_event_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.select_one('h1.page-title').text.strip()
    date_location = soup.select_one('div.medium-6.columns.event-top > h4').text.split('|')
    date = date_location[0].strip()
    location = date_location[1].strip()
    event_type = soup.select_one('a.button.big.medium.black.category').text.strip()
    return {
        'title': title,
        'date': date,
        'location': location,
        'type': event_type
    }

event_details = [get_event_details(url) for url in urls]

def get_location_coordinates(location_name):
    base_url = "https://nominatim.openstreetmap.org/search.php"
    query_params = {"q": location_name, "format": "jsonv2"}
    response = requests.get(base_url, params=query_params)
    location_data = response.json()
    if location_data:
        lat = location_data[0]['lat']
        lon = location_data[0]['lon']
        return lat, lon
    else:
        return None, None

def get_weather_forecast(lat, lon):
    url = f"https://api.weather.gov/points/{lat},{lon}"
    res = requests.get(url)
    if res.status_code != 200:
        return 'Weather data not available'
    point_dict = res.json()
    if 'properties' in point_dict and 'forecast' in point_dict['properties']:
        forecast_url = point_dict['properties']['forecast']
        forecast_res = requests.get(forecast_url)
        if forecast_res.status_code == 200:
            forecast_data = forecast_res.json()
            return forecast_data['properties']['periods'][0]['detailedForecast']
    return 'Weather data not available'

for event in event_details:
    lat, lon = get_location_coordinates(event['location'])
    if lat is None or lon is None:
        event['weather'] = 'Location coordinates not available'
    else:
        event['latitude'] = lat
        event['longitude'] = lon
        event['weather'] = get_weather_forecast(lat, lon)

csv_file_path = 'events.csv'

headers = ['title', 'date', 'location', 'type', 'latitude', 'longitude', 'weather']

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()
    for event in event_details:
        writer.writerow(event)

print(f"Data saved to {csv_file_path}")


Data saved to events.csv
